`@author: Anmol Parida`

<a href='#the_destination'>Link to the destination'</a>



<a href='#model_1'>Model_1</a>


# GestureRecognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
import imageio

In [6]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [7]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

<a id='model_2'>model_3</a>

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [8]:
# train_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/train.csv').readlines())
# val_doc = np.random.permutation(open('/notebooks/storage/Final_data/Collated_training/val.csv').readlines())
# batch_size = 20

train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 10

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [9]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [10]:
curr_dt_time = datetime.datetime.now()

train_path = 'Project_data/train'

val_path = 'Project_data/val'

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)

num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [11]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras

<a id='model_1'>Model_1</a>

In [12]:

model_1 = Sequential()

model_1.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(32, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

model_1.add(Conv3D(64, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_1.add(BatchNormalization())
model_1.add(Activation('relu'))

model_1.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_1.add(Flatten())

model_1.add(Dense(1000, activation='relu'))
model_1.add(Dropout(0.5))

model_1.add(Dense(500, activation='relu'))
model_1.add(Dropout(0.5))

#softmax layer
model_1.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [13]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_1 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_1 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_2 (Activation)    (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [14]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [15]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [16]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:
model_1.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 7s - loss: 6.0203 - categorical_accuracy: 0.2569 Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:46: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 238s 4s/step - loss: 5.9099 - categorical_accuracy: 0.2636 - val_loss: 2.4030 - val_categorical_accuracy: 0.3800

Epoch 00001: saving model to model_init_2020-05-1916_02_27.202940/model-00001-5.95587-0.25943-2.40296-0.38000.h5
Epoch 2/10
67/67 [==============================] - 25s 377ms/step - loss: 3.3178 - categorical_accuracy: 0.2886 - val_loss: 3.2601 - val_categorical_accuracy: 0.3200

Epoch 00002: saving model to model_init_2020-05-1916_02_27.202940/model-00002-3.31783-0.28856-3.26007-0.32000.h5
Epoch 3/10
67/67 [==============================] - 27s 398ms/step - loss: 2.4038 - categorical_accuracy: 0.2836 - val_loss: 1.4728 - val_categorical_accuracy: 0.4100

Epoch 00003: saving model to model_init_2020-05-1916_02_27.202940/model-00003-2.40382-0.28358-1.47278-0.41000.h5
Epoch 4/10
67/67 [==============================] - 26s 382ms/step - loss: 1.7695 - categorical_accuracy: 0.3284 - val_loss: 1.4694 - val_categorical_accuracy: 0.3100

Ep

In [17]:
classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop
channel = 3
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator_ex(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    batch_data[folder,idx] = temp #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

# [Model 2](#another_cell)

In [18]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

model_2 = Sequential()
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Flatten())
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(classes, activation='softmax'))

model_2.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 32)  2624      
_________________________________________________________________
activation_5 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 30, 120, 120, 32)  27680     
_________________________________________________________________
activation_6 (Activation)    (None, 30, 120, 120, 32)  0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 10, 40, 40, 32)    0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 40, 40, 32)    0         
_________________________________________________________________
conv3d_7 (Conv3D)            (None, 10, 40, 40, 64)    55360     
__________

In [19]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [16]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


56/67 [========================>.....] - ETA: 20s - loss: 1.8116 - categorical_accuracy: 0.1893Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 130s 2s/step - loss: 1.7785 - categorical_accuracy: 0.1881 - val_loss: 1.6095 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2020-05-1916_02_27.202940/model-00001-1.78032-0.19005-1.60946-0.21000.h5
Epoch 2/10
67/67 [==============================] - 46s 688ms/step - loss: 1.6093 - categorical_accuracy: 0.2239 - val_loss: 1.6088 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2020-05-1916_02_27.202940/model-00002-1.60933-0.22388-1.60881-0.21000.h5
Epoch 3/10
67/67 [==============================] - 46s 687ms/step - loss: 1.6128 - categorical_accuracy: 0.1393 - val_loss: 1.6081 - val_categorical_accuracy: 0.2100

Epoch 00003: saving model to model_init_2020-05-1916_02_27.202940/model-00003-1.61277-0.13930-1.60811-0.21000.h5
Epoch 4/10
67/67 [==============================] - 46s 688ms/step - loss: 1.6100 - categorical_accuracy: 0.2040 - val_loss: 1.6085 - val_categorical_accuracy: 0.2100

Ep

`Experiment 1` 
Updating the x,y,z parameters for model_2
    - number of frames(x)
    - image width(y)
    - image height(z)

In [20]:
x = 30 # number of frames
y = 60 # image width
z = 60 # image height

In [21]:
model_2 = Sequential()
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_2.add(Dropout(0.25))

model_2.add(Flatten())
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(classes, activation='softmax'))

model_2.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [22]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [20]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 1.6151 - categorical_accuracy: 0.2262Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 73s 1s/step - loss: 1.6130 - categorical_accuracy: 0.2224 - val_loss: 1.5205 - val_categorical_accuracy: 0.2300

Epoch 00001: saving model to model_init_2020-05-1916_02_27.202940/model-00001-1.61398-0.22474-1.52054-0.23000.h5
Epoch 2/10
67/67 [==============================] - 23s 343ms/step - loss: 1.5353 - categorical_accuracy: 0.3134 - val_loss: 1.4408 - val_categorical_accuracy: 0.3300

Epoch 00002: saving model to model_init_2020-05-1916_02_27.202940/model-00002-1.53535-0.31343-1.44075-0.33000.h5
Epoch 3/10
67/67 [==============================] - 25s 368ms/step - loss: 1.5121 - categorical_accuracy: 0.3134 - val_loss: 1.4274 - val_categorical_accuracy: 0.4100

Epoch 00003: saving model to model_init_2020-05-1916_02_27.202940/model-00003-1.51206-0.31343-1.42742-0.41000.h5
Epoch 4/10
67/67 [==============================] - 24s 351ms/step - loss: 1.4981 - categorical_accuracy: 0.3532 - val_loss: 1.4454 - val_categorical_accuracy: 0.3800

Epo

`Experiment 2` **Changing Batch size to 20**

In [23]:
train_generator = generator_ex(train_path, train_doc, 20)
val_generator = generator_ex(val_path, val_doc, 20)

In [22]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 20
Source path =  Project_data/train ; batch size = 20
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


32/67 [=============>................] - ETA: 1:18 - loss: 1.0276 - categorical_accuracy: 0.5703Batch:  34 Index: 20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 87s 1s/step - loss: 0.9903 - categorical_accuracy: 0.5964 - val_loss: 1.0747 - val_categorical_accuracy: 0.5500

Epoch 00001: saving model to model_init_2020-05-1916_02_27.202940/model-00001-1.01531-0.57349-1.07469-0.55000.h5
Epoch 2/10
67/67 [==============================] - 29s 431ms/step - loss: 1.0071 - categorical_accuracy: 0.5721 - val_loss: 1.0350 - val_categorical_accuracy: 0.6200

Epoch 00002: saving model to model_init_2020-05-1916_02_27.202940/model-00002-1.00707-0.57214-1.03502-0.62000.h5
Epoch 3/10
67/67 [==============================] - 30s 446ms/step - loss: 1.1225 - categorical_accuracy: 0.5323 - val_loss: 1.1373 - val_categorical_accuracy: 0.5400

Epoch 00003: saving model to model_init_2020-05-1916_02_27.202940/model-00003-1.12245-0.53234-1.13733-0.54000.h5
Epoch 4/10
67/67 [==============================] - 30s 450ms/step - loss: 1.1008 - categorical_accuracy: 0.5224 - val_loss: 1.0324 - val_categorical_accuracy: 0.5500

Epo

`Experiment 3` **Changing Batch size to 30**

In [24]:
train_generator = generator_ex(train_path, train_doc, 30)
val_generator = generator_ex(val_path, val_doc, 30)

In [24]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 30
Source path =  Project_data/train ; batch size = 30
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 3/67 [>.............................] - ETA: 4:41 - loss: 0.3835 - categorical_accuracy: 0.8667Batch:  4 Index: 30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 88s 1s/step - loss: 0.4745 - categorical_accuracy: 0.8156 - val_loss: 1.0959 - val_categorical_accuracy: 0.5813

Epoch 00001: saving model to model_init_2020-05-1916_02_27.202940/model-00001-0.48536-0.82767-1.09592-0.58125.h5
Epoch 2/10
67/67 [==============================] - 24s 364ms/step - loss: 0.5192 - categorical_accuracy: 0.8159 - val_loss: 1.2321 - val_categorical_accuracy: 0.4500

Epoch 00002: saving model to model_init_2020-05-1916_02_27.202940/model-00002-0.51925-0.81592-1.23213-0.45000.h5
Epoch 3/10
67/67 [==============================] - 25s 371ms/step - loss: 0.4631 - categorical_accuracy: 0.8259 - val_loss: 0.9725 - val_categorical_accuracy: 0.5800

Epoch 00003: saving model to model_init_2020-05-1916_02_27.202940/model-00003-0.46306-0.82587-0.97246-0.58000.h5
Epoch 4/10
67/67 [==============================] - 23s 346ms/step - loss: 0.3666 - categorical_accuracy: 0.8507 - val_loss: 0.9606 - val_categorical_accuracy: 0.6000

Epo

`Experiment 4` **Changing Batch size to 40**

In [25]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [29]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 123s 2s/step - loss: 0.2517 - categorical_accuracy: 0.9078 - val_loss: 1.1252 - val_categorical_accuracy: 0.6200

Epoch 00001: saving model to model_init_2020-05-1916_02_27.202940/model-00001-0.25174-0.90781-1.12515-0.62000.h5
Epoch 2/10
67/67 [==============================] - 121s 2s/step - loss: 0.2600 - categorical_accuracy: 0.9017 - val_loss: 1.1835 - val_categorical_accuracy: 0.6200

Epoch 00002: saving model to model_init_2020-05-1916_02_27.202940/model-00002-0.26003-0.90167-1.18351-0.62000.h5
Epoch 3/10
67/67 [==============================] - 121s 2s/step - loss: 0.2330 - categorical_accuracy: 0.9157 - val_loss: 1.1103 - val_categorical_accuracy: 0.6050

Epoch 00003: saving model to model_init_2020-05-1916_02_27.202940/model-00003-0.23302-0.91572-1.11034-0.60500.h5
Epoch 4/10
67/67 [==============================] - 123s 2s/step - loss: 0.2366 - categorical_accuracy: 0.9148 - val_loss: 1.1958 - val_categorical_accuracy: 0.6050

Epoch 00

`Experiment 5` **Change Optimizer to Adadelta**

In [26]:
model_2.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [27]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [61]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


Batch:  3 Index: 40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 184s 3s/step - loss: 1.5973 - categorical_accuracy: 0.2671 - val_loss: 1.5272 - val_categorical_accuracy: 0.2333

Epoch 00001: saving model to model_init_2020-05-1918_24_07.458406/model-00001-1.61305-0.26087-1.52718-0.23333.h5
Epoch 2/10
67/67 [==============================] - 123s 2s/step - loss: 1.3857 - categorical_accuracy: 0.4366 - val_loss: 1.2789 - val_categorical_accuracy: 0.4800

Epoch 00002: saving model to model_init_2020-05-1918_24_07.458406/model-00002-1.38799-0.43519-1.27887-0.48000.h5
Epoch 3/10
67/67 [==============================] - 122s 2s/step - loss: 1.1042 - categorical_accuracy: 0.5452 - val_loss: 0.8679 - val_categorical_accuracy: 0.6400

Epoch 00003: saving model to model_init_2020-05-1918_24_07.458406/model-00003-1.10420-0.54522-0.86792-0.64000.h5
Epoch 4/10
67/67 [==============================] - 123s 2s/step - loss: 0.8431 - categorical_accuracy: 0.6664 - val_loss: 0.8334 - val_categorical_accuracy: 0.6850

Epoch 00

Till This Model: `categorical_accuracy: 0.9947` - val_loss: 1.6844 - `val_categorical_accuracy: 0.6400`


Epoch 00010: saving model to model_init_2020-05-1918_24_07.458406/model-00010-0.01637-0.99473-1.68444-0.64000.h5

Epoch 00010: ReduceLROnPlateau reducing learning rate to 0.125.

`Experiment 6` Changing `Epoch = 20`

In [28]:
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 20


In [29]:
import keras
model_2.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_10 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_7 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [31]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [65]:
model_2.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


 2/67 [..............................] - ETA: 5:33 - loss: 0.0123 - categorical_accuracy: 1.0000Batch:  3 Index: 40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 185s 3s/step - loss: 0.2391 - categorical_accuracy: 0.9383 - val_loss: 1.5867 - val_categorical_accuracy: 0.6458

Epoch 00001: saving model to model_init_2020-05-1918_24_07.458406/model-00001-0.24091-0.93797-1.58674-0.64583.h5
Epoch 2/20
67/67 [==============================] - 125s 2s/step - loss: 0.0764 - categorical_accuracy: 0.9745 - val_loss: 1.8377 - val_categorical_accuracy: 0.6350

Epoch 00002: saving model to model_init_2020-05-1918_24_07.458406/model-00002-0.07713-0.97425-1.83772-0.63500.h5
Epoch 3/20
67/67 [==============================] - 124s 2s/step - loss: 0.0354 - categorical_accuracy: 0.9851 - val_loss: 2.2681 - val_categorical_accuracy: 0.6700

Epoch 00003: saving model to model_init_2020-05-1918_24_07.458406/model-00003-0.03543-0.98507-2.26814-0.67000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.5.
Epoch 4/20
67/67 [==============================] - 122s 2s/step - loss: 0.0153 - categorical_accuracy: 0.9947 

`categorical_accuracy: 1.0000` - val_loss: 2.1155 - `val_categorical_accuracy: 0.6700`

# Model 3

In [32]:
model_3 = Sequential()
model_3.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_3.add(Activation('relu'))
model_3.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_3.add(Dropout(0.25))

model_3.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_3.add(Activation('relu'))
model_3.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_3.add(Dropout(0.25))

model_3.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_3.add(Activation('relu'))
model_3.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_3.add(Activation('relu'))
model_3.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_3.add(Dropout(0.25))

model_3.add(Flatten())
model_3.add(Dense(512, activation='relu'))
model_3.add(BatchNormalization())
model_3.add(Dropout(0.5))
model_3.add(Dense(classes, activation='softmax'))
model_3.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_13 (Conv3D)           (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_13 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_14 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 10, 20, 20, 64)    55360     
__________

In [33]:
# Changing back batch size, images per frame, height and width of image
batch_size = 10
x = 30 # number of frames
y = 120 # image width
z = 120# image height
num_epochs = 20

In [34]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [73]:
model_3.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/valSource path =  Project_data/train ; batch size = 10
 ; batch size = 10
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


56/67 [========================>.....] - ETA: 20s - loss: 2.1001 - categorical_accuracy: 0.2911Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 132s 2s/step - loss: 2.0520 - categorical_accuracy: 0.3054 - val_loss: 4.4507 - val_categorical_accuracy: 0.2500

Epoch 00001: saving model to model_init_2020-05-1918_24_07.458406/model-00001-2.06337-0.30166-4.45073-0.25000.h5
Epoch 2/20
67/67 [==============================] - 46s 684ms/step - loss: 2.1555 - categorical_accuracy: 0.2886 - val_loss: 1.7901 - val_categorical_accuracy: 0.2700

Epoch 00002: saving model to model_init_2020-05-1918_24_07.458406/model-00002-2.15550-0.28856-1.79009-0.27000.h5
Epoch 3/20
67/67 [==============================] - 46s 681ms/step - loss: 1.8553 - categorical_accuracy: 0.2985 - val_loss: 2.0619 - val_categorical_accuracy: 0.3000

Epoch 00003: saving model to model_init_2020-05-1918_24_07.458406/model-00003-1.85534-0.29851-2.06195-0.30000.h5
Epoch 4/20
67/67 [==============================] - 46s 682ms/step - loss: 1.8289 - categorical_accuracy: 0.2985 - val_loss: 1.4122 - val_categorical_accuracy: 0.3700

Ep

# Model 4

In [35]:
input_shape=(x,y,z,channel)

nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]
# Define model
model_4 = Sequential()

model_4.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

model_4.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

model_4.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

model_4.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_4.add(BatchNormalization())
model_4.add(Activation('relu'))

model_4.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_4.add(Flatten())

model_4.add(Dense(nb_dense[0], activation='relu'))
model_4.add(Dropout(0.5))

model_4.add(Dense(nb_dense[1], activation='relu'))
model_4.add(Dropout(0.5))

#softmax layer
model_4.add(Dense(nb_dense[2], activation='softmax'))
model_4.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_4.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_19 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_6 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_19 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_20 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_7 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_20 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [36]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10

In [76]:
model_4.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  Project_data/train ; batch size = 10
 Project_data/val ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 5.0197 - categorical_accuracy: 0.2554Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 86s 1s/step - loss: 4.9494 - categorical_accuracy: 0.2538 - val_loss: 1.9697 - val_categorical_accuracy: 0.3100

Epoch 00001: saving model to model_init_2020-05-1918_24_07.458406/model-00001-4.96481-0.25641-1.96972-0.31000.h5
Epoch 2/10
67/67 [==============================] - 26s 385ms/step - loss: 2.1030 - categorical_accuracy: 0.2836 - val_loss: 1.6287 - val_categorical_accuracy: 0.3800

Epoch 00002: saving model to model_init_2020-05-1918_24_07.458406/model-00002-2.10297-0.28358-1.62865-0.38000.h5
Epoch 3/10
67/67 [==============================] - 26s 394ms/step - loss: 1.7746 - categorical_accuracy: 0.3085 - val_loss: 1.5113 - val_categorical_accuracy: 0.3300

Epoch 00003: saving model to model_init_2020-05-1918_24_07.458406/model-00003-1.77455-0.30846-1.51126-0.33000.h5
Epoch 4/10
67/67 [==============================] - 27s 406ms/step - loss: 1.6286 - categorical_accuracy: 0.3433 - val_loss: 1.1996 - val_categorical_accuracy: 0.4300

Epo

`Experiment` Updating Model with images per frame, width and height of the images

In [37]:
x = 30 # number of frames
y = 64 # image width
z = 64 # image height 

classes = 5
channel = 1

# Model 5

In [38]:
model_5 = Sequential()
model_5.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_5.add(Activation('relu'))
model_5.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_5.add(Dropout(0.25))

model_5.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_5.add(Activation('relu'))
model_5.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_5.add(Dropout(0.25))

model_5.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_5.add(Activation('relu'))
model_5.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_5.add(Activation('relu'))
model_5.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_5.add(Dropout(0.25))

model_5.add(Flatten())
model_5.add(Dense(512, activation='relu'))
model_5.add(BatchNormalization())
model_5.add(Dropout(0.5))
model_5.add(Dense(classes, activation='softmax'))
model_5.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_5.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_23 (Conv3D)           (None, 30, 64, 64, 32)    896       
_________________________________________________________________
activation_23 (Activation)   (None, 30, 64, 64, 32)    0         
_________________________________________________________________
conv3d_24 (Conv3D)           (None, 30, 64, 64, 32)    27680     
_________________________________________________________________
activation_24 (Activation)   (None, 30, 64, 64, 32)    0         
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 10, 22, 22, 32)    0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 10, 22, 22, 32)    0         
_________________________________________________________________
conv3d_25 (Conv3D)           (None, 10, 22, 22, 64)    55360     
__________

In [39]:
batch_size = 5
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 20

In [80]:
model_5.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 5
Source path =  Project_data/train ; batch size = 5
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 36s 532ms/step - loss: 2.1338 - categorical_accuracy: 0.2806 - val_loss: 1.8925 - val_categorical_accuracy: 0.2400

Epoch 00001: saving model to model_init_2020-05-1918_24_07.458406/model-00001-2.13382-0.28060-1.89250-0.24000.h5
Epoch 2/20
64/67 [===========================>..] - ETA: 1s - loss: 2.0075 - categorical_accuracy: 0.2656Batch:  133 Index: 5


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 35s 529ms/step - loss: 2.0455 - categorical_accuracy: 0.2637 - val_loss: 5.0769 - val_categorical_accuracy: 0.2600

Epoch 00002: saving model to model_init_2020-05-1918_24_07.458406/model-00002-2.03819-0.26284-5.07690-0.26000.h5
Epoch 3/20
67/67 [==============================] - 22s 326ms/step - loss: 1.9711 - categorical_accuracy: 0.2189 - val_loss: 1.7188 - val_categorical_accuracy: 0.2800

Epoch 00003: saving model to model_init_2020-05-1918_24_07.458406/model-00003-1.97108-0.21891-1.71881-0.28000.h5
Epoch 4/20
67/67 [==============================] - 21s 319ms/step - loss: 1.8430 - categorical_accuracy: 0.2587 - val_loss: 2.7333 - val_categorical_accuracy: 0.1400

Epoch 00004: saving model to model_init_2020-05-1918_24_07.458406/model-00004-1.84297-0.25871-2.73330-0.14000.h5
Epoch 5/20
67/67 [==============================] - 22s 335ms/step - loss: 1.8633 - categorical_accuracy: 0.2886 - val_loss: 1.8747 - val_categorical_accuracy: 0.1400



# Model 6

In [40]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 1
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

In [41]:
model_6 = Sequential()

model_6.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_6.add(BatchNormalization())
model_6.add(Activation('relu'))

model_6.add(MaxPooling3D(pool_size=(2,2,2)))

model_6.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_6.add(BatchNormalization())
model_6.add(Activation('relu'))

model_6.add(MaxPooling3D(pool_size=(2,2,2)))

model_6.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_6.add(BatchNormalization())
model_6.add(Activation('relu'))

model_6.add(MaxPooling3D(pool_size=(2,2,2)))

model_6.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_6.add(BatchNormalization())
model_6.add(Activation('relu'))

model_6.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_6.add(Flatten())

model_6.add(Dense(nb_dense[0], activation='relu'))
model_6.add(Dropout(0.5))

model_6.add(Dense(nb_dense[1], activation='relu'))
model_6.add(Dropout(0.5))

#softmax layer
model_6.add(Dense(nb_dense[2], activation='softmax'))
model_6.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_6.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_29 (Conv3D)           (None, 30, 120, 120, 8)   224       
_________________________________________________________________
batch_normalization_11 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_29 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_19 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_30 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_12 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_30 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [42]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 30

In [84]:
model_6.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 6.1446 - categorical_accuracy: 0.2677Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 77s 1s/step - loss: 5.9941 - categorical_accuracy: 0.2741 - val_loss: 3.5396 - val_categorical_accuracy: 0.3100

Epoch 00001: saving model to model_init_2020-05-1918_24_07.458406/model-00001-6.05327-0.26998-3.53959-0.31000.h5
Epoch 2/30
67/67 [==============================] - 23s 339ms/step - loss: 2.6959 - categorical_accuracy: 0.2637 - val_loss: 1.9966 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to model_init_2020-05-1918_24_07.458406/model-00002-2.69586-0.26368-1.99657-0.21000.h5
Epoch 3/30
67/67 [==============================] - 24s 354ms/step - loss: 2.0232 - categorical_accuracy: 0.2687 - val_loss: 1.3049 - val_categorical_accuracy: 0.4900

Epoch 00003: saving model to model_init_2020-05-1918_24_07.458406/model-00003-2.02316-0.26866-1.30489-0.49000.h5
Epoch 4/30
67/67 [==============================] - 26s 387ms/step - loss: 1.7197 - categorical_accuracy: 0.3483 - val_loss: 1.3839 - val_categorical_accuracy: 0.3700

Epo


Epoch 00029: saving model to model_init_2020-05-1918_24_07.458406/model-00029-0.66864-0.72139-0.65766-0.74000.h5
Epoch 30/30
67/67 [==============================] - 25s 367ms/step - loss: 0.7189 - categorical_accuracy: 0.7214 - val_loss: 0.6495 - val_categorical_accuracy: 0.7600

Epoch 00030: saving model to model_init_2020-05-1918_24_07.458406/model-00030-0.71887-0.72139-0.64945-0.76000.h5


# Model 7

In [43]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_7 = Sequential()

model_7.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_7.add(BatchNormalization())
model_7.add(Activation('relu'))

model_7.add(MaxPooling3D(pool_size=(2,2,2)))

model_7.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_7.add(BatchNormalization())
model_7.add(Activation('relu'))

model_7.add(MaxPooling3D(pool_size=(2,2,2)))

model_7.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_7.add(BatchNormalization())
model_7.add(Activation('relu'))

model_7.add(MaxPooling3D(pool_size=(2,2,2)))

model_7.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_7.add(BatchNormalization())
model_7.add(Activation('relu'))

model_7.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_7.add(Flatten())

model_7.add(Dense(nb_dense[0], activation='relu'))
model_7.add(Dropout(0.5))

model_7.add(Dense(nb_dense[1], activation='relu'))
model_7.add(Dropout(0.5))

#softmax layer
model_7.add(Dense(nb_dense[2], activation='softmax'))
model_7.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_7.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_33 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_15 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_33 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_23 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_34 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_16 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_34 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [44]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10

In [89]:
model_7.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 7.2454 - categorical_accuracy: 0.2308Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 81s 1s/step - loss: 7.1410 - categorical_accuracy: 0.2318 - val_loss: 8.4426 - val_categorical_accuracy: 0.3000

Epoch 00001: saving model to model_init_2020-05-1918_24_07.458406/model-00001-7.17247-0.23077-8.44260-0.30000.h5
Epoch 2/10
67/67 [==============================] - 25s 379ms/step - loss: 2.9425 - categorical_accuracy: 0.2935 - val_loss: 2.8849 - val_categorical_accuracy: 0.3000

Epoch 00002: saving model to model_init_2020-05-1918_24_07.458406/model-00002-2.94250-0.29353-2.88487-0.30000.h5
Epoch 3/10
67/67 [==============================] - 26s 394ms/step - loss: 2.1480 - categorical_accuracy: 0.2289 - val_loss: 1.1879 - val_categorical_accuracy: 0.5300

Epoch 00003: saving model to model_init_2020-05-1918_24_07.458406/model-00003-2.14803-0.22886-1.18794-0.53000.h5
Epoch 4/10
67/67 [==============================] - 28s 419ms/step - loss: 1.5483 - categorical_accuracy: 0.3980 - val_loss: 1.4641 - val_categorical_accuracy: 0.3400

Epo

# Model 8

In [45]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

In [46]:
# Define model
model_8 = Sequential()

model_8.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_8.add(BatchNormalization())
model_8.add(Activation('relu'))

model_8.add(MaxPooling3D(pool_size=(2,2,2)))

model_8.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_8.add(BatchNormalization())
model_8.add(Activation('relu'))

model_8.add(MaxPooling3D(pool_size=(2,2,2)))

model_8.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_8.add(BatchNormalization())
model_8.add(Activation('relu'))

model_8.add(MaxPooling3D(pool_size=(2,2,2)))

model_8.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_8.add(BatchNormalization())
model_8.add(Activation('relu'))

model_8.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_8.add(Flatten())

model_8.add(Dense(nb_dense[0], activation='relu'))
model_8.add(Dropout(0.5))

model_8.add(Dense(nb_dense[1], activation='relu'))
model_8.add(Dropout(0.5))

#softmax layer
model_8.add(Dense(nb_dense[2], activation='softmax'))
model_8.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_8.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_37 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_19 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_37 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_27 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_38 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_20 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_38 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [47]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10

In [92]:
model_8.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 6.8387 - categorical_accuracy: 0.2569Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 85s 1s/step - loss: 6.7155 - categorical_accuracy: 0.2553 - val_loss: 3.1743 - val_categorical_accuracy: 0.3200

Epoch 00001: saving model to model_init_2020-05-1918_24_07.458406/model-00001-6.75471-0.25792-3.17435-0.32000.h5
Epoch 2/10
67/67 [==============================] - 24s 362ms/step - loss: 2.9162 - categorical_accuracy: 0.2836 - val_loss: 2.1628 - val_categorical_accuracy: 0.2800

Epoch 00002: saving model to model_init_2020-05-1918_24_07.458406/model-00002-2.91621-0.28358-2.16280-0.28000.h5
Epoch 3/10
67/67 [==============================] - 30s 445ms/step - loss: 1.9056 - categorical_accuracy: 0.3234 - val_loss: 1.2690 - val_categorical_accuracy: 0.4100

Epoch 00003: saving model to model_init_2020-05-1918_24_07.458406/model-00003-1.90559-0.32338-1.26898-0.41000.h5
Epoch 4/10
67/67 [==============================] - 26s 394ms/step - loss: 1.6231 - categorical_accuracy: 0.3632 - val_loss: 1.1548 - val_categorical_accuracy: 0.4900

Epo

# Model 9

In [48]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

In [49]:
# Define model
model_9 = Sequential()

model_9.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_9.add(BatchNormalization())
model_9.add(Activation('relu'))

model_9.add(MaxPooling3D(pool_size=(2,2,2)))

model_9.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_9.add(BatchNormalization())
model_9.add(Activation('relu'))

model_9.add(MaxPooling3D(pool_size=(2,2,2)))

model_9.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_9.add(BatchNormalization())
model_9.add(Activation('relu'))

model_9.add(MaxPooling3D(pool_size=(2,2,2)))

model_9.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_9.add(Activation('relu'))
model_9.add(Dropout(0.25))

model_9.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_9.add(Flatten())

model_9.add(Dense(nb_dense[0], activation='relu'))
model_9.add(Dropout(0.5))

model_9.add(Dense(nb_dense[1], activation='relu'))
model_9.add(Dropout(0.5))

#softmax layer
model_9.add(Dense(nb_dense[2], activation='softmax'))
model_9.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_9.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_41 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_23 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_41 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_31 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_42 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_24 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_42 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [50]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10

In [96]:
model_9.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  Project_data/train ; batch size = 10
Epoch 1/10
 Project_data/val ; batch size = 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


65/67 [============================>.] - ETA: 2s - loss: 1.7474 - categorical_accuracy: 0.3169Batch:  67 Index: 10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:44: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:48: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.


67/67 [==============================] - 86s 1s/step - loss: 1.7418 - categorical_accuracy: 0.3120 - val_loss: 1.2605 - val_categorical_accuracy: 0.3800

Epoch 00001: saving model to model_init_2020-05-1918_24_07.458406/model-00001-1.74460-0.31523-1.26046-0.38000.h5
Epoch 2/10
67/67 [==============================] - 25s 368ms/step - loss: 1.5283 - categorical_accuracy: 0.3731 - val_loss: 1.3934 - val_categorical_accuracy: 0.3300

Epoch 00002: saving model to model_init_2020-05-1918_24_07.458406/model-00002-1.52830-0.37313-1.39340-0.33000.h5
Epoch 3/10
67/67 [==============================] - 27s 399ms/step - loss: 1.4790 - categorical_accuracy: 0.3532 - val_loss: 1.1598 - val_categorical_accuracy: 0.4900

Epoch 00003: saving model to model_init_2020-05-1918_24_07.458406/model-00003-1.47896-0.35323-1.15983-0.49000.h5
Epoch 4/10
67/67 [==============================] - 27s 406ms/step - loss: 1.4308 - categorical_accuracy: 0.4080 - val_loss: 1.5231 - val_categorical_accuracy: 0.4900

Epo

Epoch 10/10 `categorical_accuracy: 0.6866` - `val_categorical_accuracy: 0.6900`

Epoch 00010: saving model to model_init_2020-05-1918_24_07.458406/model-00010-0.81188-0.68657-0.80880-0.69000.h5

# Model 10  `Final Model`

In [51]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(30,120,120,3)

In [52]:
# Define model
model_final = Sequential()

model_final.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(Activation('relu'))
model_final.add(Dropout(0.25))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_final.add(Flatten())

model_final.add(Dense(nb_dense[0], activation='relu'))
model_final.add(Dropout(0.5))

model_final.add(Dense(nb_dense[1], activation='relu'))
model_final.add(Dropout(0.5))

#softmax layer
model_final.add(Dense(nb_dense[2], activation='softmax'))

In [53]:
optimiser = keras.optimizers.Adam() #write your optimizer
model_final.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_final.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_45 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_26 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_45 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_35 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_46 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_27 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_46 (Activation)   (None, 15, 60, 60, 16)    0         
__________

In [54]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [55]:
batch_size = 10
num_epochs = 20
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 10
Source path =  Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 228s 3s/step - loss: 1.9131 - categorical_accuracy: 0.2642 - val_loss: 1.3580 - val_categorical_accuracy: 0.3500

Epoch 00001: saving model to model_init_2020-05-2007_01_46.608464/model-00001-1.91648-0.26697-1.35803-0.35000.h5
Epoch 2/20
67/67 [==============================] - 27s 399ms/step - loss: 1.4457 - categorical_accuracy: 0.3483 - val_loss: 1.5636 - val_categorical_accuracy: 0.2300

Epoch 00002: saving model to model_init_2020-05-2007_01_46.608464/model-00002-1.44568-0.34826-1.56359-0.23000.h5
Epoch 3/20
67/67 [==============================] - 28s 421ms/step - loss: 1.5339 - categorical_accuracy: 0.2886 - val_loss: 1.3649 - val_categorical_accuracy: 0.4700

Epoch 00003: saving model to model_init_2020-05-2007_01_46.608464/model-00003-1.53390-0.28856-1.36486-0.47000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to

In [56]:
batch_size = 32
num_epochs = 20
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20
67/67 [==============================] - 29s 425ms/step - loss: 0.9019 - categorical_accuracy: 0.6318 - val_loss: 0.8696 - val_categorical_accuracy: 0.6500

Epoch 00001: saving model to model_init_2020-05-2007_01_46.608464/model-00001-0.90194-0.63184-0.86959-0.65000.h5
Epoch 2/20
67/67 [==============================] - 28s 416ms/step - loss: 0.9171 - categorical_accuracy: 0.5622 - val_loss: 0.7775 - val_categorical_accuracy: 0.7600

Epoch 00002: saving model to model_init_2020-05-2007_01_46.608464/model-00002-0.91714-0.56219-0.77747-0.76000.h5
Epoch 3/20
67/67 [==============================] - 27s 403ms/step - loss: 0.9289 - categorical_accuracy: 0.6269 - val_loss: 0.8008 - val_categorical_accuracy: 0.7500

Epoch 00003: saving model to model_init_2020-05-2007_01_46.608464/model-00003-0.92889-0.62687-0.80076-0.75000.h5
Epoch 4/20
67/67 [==============================] - 26s 392ms/step - loss: 0.8536 - categorical_accuracy: 0.6617 - val_loss: 0.8708 - val_categorical_accurac

In [57]:
batch_size = 64
num_epochs = 20

train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)


model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path = Source path =  Project_data/train  Project_data/val ; batch size = 64
; batch size = 64
Epoch 1/20
Batch:  2 Index: 64
67/67 [==============================] - 238s 4s/step - loss: 0.8160 - categorical_accuracy: 0.6739 - val_loss: 0.8086 - val_categorical_accuracy: 0.6899

Epoch 00001: saving model to model_init_2020-05-2007_01_46.608464/model-00001-0.80642-0.67374-0.80856-0.68987.h5
Epoch 2/20
67/67 [==============================] - 137s 2s/step - loss: 0.7795 - categorical_accuracy: 0.6863 - val_loss: 0.8151 - val_categorical_accuracy: 0.7300

Epoch 00002: saving model to model_init_2020-05-2007_01_46.608464/model-00002-0.77896-0.68690-0.81509-0.73000.h5
Epoch 3/20
67/67 [==============================] - 138s 2s/step - loss: 0.7890 - categorical_accuracy: 0.6901 - val_loss: 0.6994 - val_categorical_accuracy: 0.6750

Epoch 00003: saving model to model_init_2020-05-2007_01_46.608464/model-00003-0.78898-0.69008-0.69939-0.67500.h5
Epoch 4/20
67/67 [=======================

In [58]:
batch_size = 40
num_epochs = 20

train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)


model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/20
67/67 [==============================] - 216s 3s/step - loss: 0.7638 - categorical_accuracy: 0.6972 - val_loss: 0.7692 - val_categorical_accuracy: 0.7250

Epoch 00001: saving model to model_init_2020-05-2007_01_46.608464/model-00001-0.76342-0.69681-0.76920-0.72500.h5
Epoch 2/20
67/67 [==============================] - 148s 2s/step - loss: 0.7920 - categorical_accuracy: 0.6779 - val_loss: 0.8121 - val_categorical_accuracy: 0.6950

Epoch 00002: saving model to model_init_2020-05-2007_01_46.608464/model-00002-0.79257-0.67725-0.81206-0.69500.h5
Epoch 3/20
67/67 [==============================] - 143s 2s/step - loss: 0.7925 - categorical_accuracy: 0.6857 - val_loss: 0.8079 - val_categorical_accuracy: 0.7100

Epoch 00003: saving model to model_init_2020-05-2007_01_46.608464/model-00003-0.79250-0.68569-0.80795-0.71000.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 3.8

`Code till 19-May-2020` `To Be Continued` 